In [135]:
import numpy as np
from sklearn.datasets import make_classification
from DMC_class import * 
import time
from sklearn.cluster import KMeans
import pandas as pd
k=50
t=30
n_samples=10000
X, y = make_classification(n_features=15,n_samples=n_samples,random_state=42,n_classes=k,n_informative=8)
XD=KMeans(n_clusters=t).fit(X).labels_
def checkarray(A,B):
    bandera=0
    for i in range(len(A)):
        if np.abs(A[i]-B[i])>0.01:
            print(A[i],B[i])
            bandera=1
            print("error")
    if bandera==0:
        print("ok")

def checkMatriz(A,B):
    bandera=0
    for i in range(len(A)):
        for j in range(len(A[0])):
            if A[i,j]!=B[i,j]:
                bandera=1
                print("errror")
    if bandera==0:
        print("ok")
XD


array([ 9,  9, 16, ..., 11, 22,  2])

In [136]:
y

array([26, 24, 30, ...,  4,  9, 40])

### compute_pi

In [139]:
def compute_pHatOrginal(XD: np.ndarray, y: np.ndarray, K: int, T: int):
    """
    Parameters
    ----------
    XD : ndarray of shape (n_samples,)
        Labels of profiles for each data point

    y : ndarray of shape (n_samples,)
        Labels

    K : int
        Number of classes

    T : int
        Number of profiles

    Returns
    -------
    pHat : ndarray of shape(K, n_profiles)
    """
    pHat = np.zeros((K, T))

    for k in range(K):
        Ik = np.where(y == k)[0]
        mk = len(Ik)
        for t in range(T):
            pHat[k, t] = np.sum(XD[Ik] == t) / mk
    return pHat

def compute_pHatOpti(XD: np.ndarray, y: np.ndarray, K: int, T: int):
    """
    Parameters
    ----------
    XD : ndarray of shape (n_samples,)
        Labels of profiles for each data point

    y : ndarray of shape (n_samples,)
        Labels

    K : int
        Number of classes

    T : int
        Number of profiles

    Returns
    -------
    pHat : ndarray of shape(K, n_profiles)
    """
    pHat = np.zeros((K, T))

    for k in range(K):
        Ik = np.where(y == k)[0]
        mk = len(Ik)
        
        pHat[k] = np.bincount(XD[Ik], minlength=T)/mk
    return pHat

start=time.time()
B=compute_pHatOrginal(XD,y,k,t)
print(time.time()-start)
start=time.time()
A=compute_pHatOpti(XD,y,k,t)
print(time.time()-start)
checkMatriz(A,B)

0.011998176574707031
0.0010004043579101562
ok


In [142]:
from sklearn.metrics import confusion_matrix
y_pred=np.random.randint(0, k, size=n_samples)
L=np.ones((k, k)) - np.eye(k)

def compute_conditional_riskWenlong(y_true: np.ndarray, y_pred: np.ndarray, K: int, L: np.ndarray):
    """
    Parameters
    ----------
    y_true : ndarray of shape (n_samples,)
        Real labels

    y_pred : ndarray of shape (n_samples,)
        Predicted labels

    K : int
        Number of classes

    L : ndarray of shape (K, K)
        Loss matrix, where K is the number of unique classes

    Returns
    -------
    R : ndarray of shape (K,)
        Conditional risk

    confmat : ndarray of shape (K, K)
        Confusion matrix
    """
    # Identify all unique classes
    unique_classes = np.unique(np.concatenate((y_true, y_pred)))
    class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}

    # Initialize the confusion matrix
    confmat = np.zeros((K, K))

    # Populate the confusion matrix
    for true_class in unique_classes:
        true_class_index = class_to_index[true_class]
        Ik = np.where(y_true == true_class)[0]
        pred_classes_indices = [class_to_index[pred] for pred in y_pred[Ik]]
        for pred_index in pred_classes_indices:
            confmat[true_class_index, pred_index] += 1

    # Normalize the rows of the confusion matrix to get probabilities
    row_sums = confmat.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1  # Prevent division by zero for classes not in y_true
    confmat /= row_sums

    # Calculate the conditional risk for each true category
    R = np.dot(L, confmat.T).diagonal()

    return R, confmat

def compute_conditional_riskSalva(YR, Yhat, K, L): 
    '''
    Function to compute the class-conditional risks.
    Parameters
    ----------
    YR : DataFrame
        Real labels.
    Yhat : Array
        Predicted labels.
    K : int
        Number of classes.
    L : Array
        Loss Function.

    Returns
    -------
    R : Array of floats
        Conditional risks.
    confmat : Matrix
        Confusion matrix.
    '''
    Labels=[i for i in range(K)]
    confmat=confusion_matrix(np.array(YR),np.array(Yhat),normalize='true',labels=Labels)
    R=np.sum(np.multiply(L, confmat),axis=1)

   # Is only the confuns 
    
    return R, confmat

def compute_conditional_riskCyprien(YR, Yhat, K, L): 
    '''
    Parameters
    ----------
    YR : DataFrame
        Real labels.
    Yhat : Array
        Predicted labels.
    K : int
        Number of classes.
    L : Array
        Loss Function.

    Returns
    -------
    R : Array of floats
        Conditional risks.
    confmat : Matrix
        Confusion matrix.
    '''
   
    confmat = np.zeros((K, K))
    R = np.zeros((1, K))
    YR_liste= YR.values.tolist()
    for k in range(0, K):
        mk = YR_liste.count([k+1])
        if mk > 0:
            Ik = np.where(YR == (k + 1))
            for l in range(0, K):
                confmat[k,l] = sum(Yhat[Ik[0]]==l+1)/mk
        R[0,k] = L[k, :].dot(confmat[k, :]) 
    
    return R, confmat
start=time.time()
O1,O2=compute_conditional_riskCyprien(pd.DataFrame(y+1),y_pred+1,k,L)
print(time.time()-start)
start=time.time()
W1,W2=compute_conditional_riskWenlong(y,y_pred,k,L)
print(time.time()-start)
start=time.time()
S1,S2=compute_conditional_riskSalva(y,y_pred,k,L)
print(time.time()-start)
checkMatriz(S2,W2)
checkarray(S1,W1)


0.8189289569854736
0.005002260208129883
0.0019969940185546875
ok
ok
